# 1. Import Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

# 2. Create Functions

### Get links

In [5]:
def get_product_links_fendi_femme_newin(url):
    # Set options to run headless (without opening a browser window)
    options = Options()
    options.headless = True

    # Initialize a browser
    driver = webdriver.Firefox(options=options)

    # Open the webpage
    driver.get(url)

    # Wait for the "Voir ... de plus" button to become clickable, and click it
    wait = WebDriverWait(driver, 30)
    while True:
        try:
            load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "show-more") and contains(text(), "Voir")]')))
            load_more_button.click()
            # Wait for the page to load
            time.sleep(30)
        except Exception as e:
            print("Reached the end of the page or an error occurred:", e)
            break

    # Find all product links
    #product_links = [a.get_attribute('href') for a in driver.find_elements_by_xpath('//div[contains(@class, "product")]//a')]
    product_links = [a.get_attribute('href') for a in driver.find_elements(by=By.XPATH, value='//div[@class="product"]//a')]
    

    # Close the browser
    driver.quit()

    return product_links

In [6]:
# Test
url = "https://www.fendi.com/fr-fr/femme/new-in"
link = get_product_links_fendi_femme_newin(url)
link

Reached the end of the page or an error occurred: Message: Element <button class="c-search-results__product-grid-show-more show-more btn btn-primary btn-primary--small-pd-ar more"> is not clickable at point (639,712) because another element <div class="onetrust-pc-dark-filter ot-fade-in"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:335:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31



['https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql',
 'https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/robe-robe-en-coton-et-soie-bleu-clair-fzdc49arv4f08s8',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/robe-robe-en-coton-et-soie-bleu-clair-fzdc49arv4f08s8',
 'https://www.fendi.com/fr-fr/femme/new-in/bracelet-forever-fendi-bracelet-en-cuir-rouge-8ak630aqp4f1o8x',
 'https://www.fendi.com/fr-fr/femme/new-in/bracelet-forever-fendi-bracelet-en-cuir-rouge-8ak630aqp4f1o8x',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-sandales-a-talon-aiguille-en-cuir-beige-8x8499tc3f0vc9',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-sandales-a-talon-aiguille-en-cuir-beige-8x8499tc3f0vc9',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/haut-bustier-en-laine-multicolore-fzxb21ar

In [7]:
len(link)

48

In [5]:
# Keep for reference

from bs4 import BeautifulSoup
import requests

def get_product_links_soup(url):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all product divs
    products = soup.find_all('div', {'class': 'product'})

    # Extract the href attribute from the <a> tag within each product div
    links = [product.find('a')['href'] for product in products]

    return links

In [8]:
# Test
url = "https://www.fendi.com/fr-fr/femme/new-in"
link = get_product_links_soup(url)
link

['https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/robe-robe-en-coton-et-soie-bleu-clair-fzdc49arv4f08s8',
 'https://www.fendi.com/fr-fr/femme/new-in/bracelet-forever-fendi-bracelet-en-cuir-rouge-8ak630aqp4f1o8x',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-sandales-a-talon-aiguille-en-cuir-beige-8x8499tc3f0vc9',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/haut-bustier-en-laine-multicolore-fzxb21arv7f1p2d',
 'https://www.fendi.com/fr-fr/femme/sacs/sacs-exotiques/sac-fendi-origami-mini-mini-sac-en-karung-naturel-8bs083arbsf1nxk',
 'https://www.fendi.com/fr-fr/femme/new-in/jupe-jupe-courte-en-laine-multicolore-fzq738arv7f1p2c',
 'https://www.fendi.com/fr-fr/femme/new-in/bijou-de-sac-fendi-fortune-teller-bijou-de-sac-en-nappa-rouge-7as286k47f1ozt',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-bottes-a-talon-moyen-en-karung-beige-8w85

In [9]:
len(link)

12

In [27]:
# Keep for reference

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def get_product_links_roll(url):
    # Set options to run headless (without opening a browser window)
    options = Options()
    options.headless = True

    # Initialize a browser
    driver = webdriver.Firefox(options=options)

    # Open the webpage
    driver.get(url)

    # Wait for the "Voir ... de plus" button to become clickable, and click it
    wait = WebDriverWait(driver, 15)
    while True:
        try:
            load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "show-more") and contains(text(), "Voir")]')))
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            # Wait for the page to load
            time.sleep(15)
        except Exception as e:
            print("Reached the end of the page or an error occurred:", e)
            break

    # Find all product links
    #product_links = [a.get_attribute('href') for a in driver.find_elements(By.XPATH, '//div[@class="product"]//a')]
    product_links = [a.get_attribute('href') for a in driver.find_elements(by=By.XPATH, value='//div[@class="product"]//a')]

    # Close the browser
    driver.quit()

    return product_links

In [28]:
# Test
url = "https://www.fendi.com/fr-fr/femme/new-in?start=0&sz=24"
link = get_product_links_roll(url)
link

Reached the end of the page or an error occurred: Message: Element <button class="c-search-results__product-grid-show-more show-more btn btn-primary btn-primary--small-pd-ar more"> is not clickable at point (639,22) because another element <div class="onetrust-pc-dark-filter ot-fade-in"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:335:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31



['https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql',
 'https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/robe-robe-en-coton-et-soie-bleu-clair-fzdc49arv4f08s8',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/robe-robe-en-coton-et-soie-bleu-clair-fzdc49arv4f08s8',
 'https://www.fendi.com/fr-fr/femme/new-in/bracelet-forever-fendi-bracelet-en-cuir-rouge-8ak630aqp4f1o8x',
 'https://www.fendi.com/fr-fr/femme/new-in/bracelet-forever-fendi-bracelet-en-cuir-rouge-8ak630aqp4f1o8x',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-sandales-a-talon-aiguille-en-cuir-beige-8x8499tc3f0vc9',
 'https://www.fendi.com/fr-fr/femme/chaussures/fendi-filo-sandales-a-talon-aiguille-en-cuir-beige-8x8499tc3f0vc9',
 'https://www.fendi.com/fr-fr/femme/pret-a-porter/haut-bustier-en-laine-multicolore-fzxb21ar

In [29]:
len(link)

48

### Get ID, Name, Price

In [9]:
from bs4 import BeautifulSoup
import requests
import re
import json
def get_product_info_fendi_femme_newin(url):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the script tag containing the product info
    script = soup.find('script', string=re.compile('window.pdpProduct'))

    # Extract the JavaScript code from the script tag
    js_code = script.string

    # Use a regular expression to match the product info
    match = re.search(r'window.pdpProduct = (\[.*?\])', js_code)

    # Parse the product info as JSON
    product_info = json.loads(match.group(1))

    # Extract the product ID, name, and price
    product_id = product_info[0]['id']
    product_name = product_info[0]['name']
    product_price = product_info[0]['price']

    # Create a DataFrame from the product info
    product_dict = {
        "ID": product_id,
        "Name": product_name,
        "Price": product_price
    }

    return product_dict

url = "https://www.fendi.com/fr-fr/femme/sacs/sacs-cabas/fendi-flip-medium-sac-transformable-en-cuir-color-block-8bh419arbjf1oql"
print(get_product_info_fendi_femme_newin(url))

{'ID': '8BH419ARBJF1OQL', 'Name': 'Fendi Flip Medium', 'Price': '1500.00'}


In [ ]:
 #product_info = {}
  #  if product_id:
   #     product_info['ID'] = product_id
    #if product_name:
     #   product_info['Name'] = product_name
    #if product_price:
     #   product_info['Price'] = product_price    
    
    #return product_info

### Get Full info

In [16]:
def get_all_product_info_fendi_femme_newin(url):
    # List to store product information
    product_info_list = []

    # Get product information for each product
    
    product_links = get_product_links_fendi_femme_newin(url)
    for link in product_links:
        product_info = get_product_info_fendi_femme_newin(link)
        product_info_list.append(product_info)

    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(product_info_list)

    return df

In [17]:
# Test
url = "https://www.fendi.com/fr-fr/femme/new-in"
test_info = get_all_product_info_fendi_femme_newin(url)
test_info

Reached the end of the page or an error occurred: Message: Element <button class="c-search-results__product-grid-show-more show-more btn btn-primary btn-primary--small-pd-ar more"> is not clickable at point (639,712) because another element <div class="onetrust-pc-dark-filter ot-fade-in"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:335:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:176:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:135:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:84:31



,ID,Name,Price
0,8BH419ARBJF1OQL,Fendi Flip Medium,1500.00
1,8BH419ARBJF1OQL,Fendi Flip Medium,1500.00
2,FZDC49ARV4F08S8,Robe,2900.00
3,FZDC49ARV4F08S8,Robe,2900.00
4,8AK630AQP4F1O8X,Bracelet Forever Fendi,290.00
5,8AK630AQP4F1O8X,Bracelet Forever Fendi,290.00
6,8X8499TC3F0VC9,Fendi Filo,790.00
7,8X8499TC3F0VC9,Fendi Filo,790.00
8,FZXB21ARV7F1P2D,Haut,1200.00
9,FZXB21ARV7F1P2D,Haut,1200.00


In [18]:
test_info.to_excel('fendi_femme_newin.xlsx')